<a href="https://colab.research.google.com/github/govindrathore27/transfer_learning_pneumonia/blob/main/Transfer_Learning_pneumonia.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q kaggle

In [ ]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"govindsrathore","key":"f85d4f6feb7a1535183898e2c3fa59a0"}'}

In [ ]:
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/

mkdir: cannot create directory ‘/root/.kaggle’: File exists


In [ ]:
!chmod 600  ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d paultimothymooney/chest-xray-pneumonia

100% 2.28G/2.29G [00:29<00:00, 34.3MB/s]
100% 2.29G/2.29G [00:30<00:00, 81.8MB/s]


In [ ]:
from keras.applications import VGG16
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from keras.preprocessing.image import ImageDataGenerator
import keras
import os
import zipfile

In [ ]:
!mkdir chest_xray_pneumonia

In [ ]:
!ls

chest_xray_pneumonia  chest-xray-pneumonia.zip	kaggle.json  sample_data


In [ ]:
with zipfile.ZipFile("chest-xray-pneumonia.zip","r") as z:
    z.extractall("chest_xray_pneumonia")

In [ ]:
main_path = "chest_xray_pneumonia/chest_xray"

In [ ]:
train_path = os.path.join(main_path,'train')
test_path=os.path.join(main_path,"test")
val_path=os.path.join(main_path,"val")

In [ ]:
train_Datagen = ImageDataGenerator(
    rescale =1/255,
    shear_range=10,
    zoom_range = 0.2,
    horizontal_flip = True,
    width_shift_range=0.2,
    height_shift_range=0.2,
    rotation_range=40,
    fill_mode = 'nearest',
)
val_datagen = ImageDataGenerator(
    rescale = 1/255
)

In [ ]:
conv_base = VGG16(include_top = False,weights='imagenet')
for i in conv_base.layers:
    i.trainable=False
X = conv_base.output
X = keras.layers.GlobalAveragePooling2D()(X)
X = keras.layers.Dense(128,activation='relu')(X)
predictions = keras.layers.Dense(1,activation='sigmoid')(X)
model= keras.Model(conv_base.input,predictions)
model.compile(loss='binary_crossentropy',
             optimizer='rmsprop',
             metrics=['accuracy'])

In [ ]:
train_generator=train_Datagen.flow_from_directory(
    train_path,
    target_size=(150,150),
    batch_size=32,
    class_mode='binary'
)
validation_generator = val_datagen.flow_from_directory(
        val_path,
        target_size=(150, 150),
        batch_size=16,
        class_mode='binary'
)
test_generator = val_datagen.flow_from_directory(
    test_path,
    target_size=(150,150),
    batch_size=32,
    class_mode='binary'
)

Found 5216 images belonging to 2 classes.
Found 16 images belonging to 2 classes.
Found 624 images belonging to 2 classes.


In [ ]:
model.fit_generator(
    train_generator,
    steps_per_epoch = 50,
    epochs=100,
    validation_data = validation_generator
)

In [ ]:
accuracy=model.evaluate_generator(test_generator,624)[1]

In [ ]:
accuracy

0.8862179517745972

In [ ]:
#unfreeze to fine-tune the model and increase the accuracy
for i in conv_base.layers:
  i.trainable = True


In [ ]:
model.compile(optimizer=keras.optimizers.Adam(1e-5),  # Very low learning rate
              loss=keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=[keras.metrics.BinaryAccuracy()])

In [ ]:
model.fit_generator(
    train_generator,
    steps_per_epoch = 50,
    epochs=100,
    validation_data = validation_generator
)

50/50 [==============================] - 33s 663ms/step - loss: 0.4295 - binary_accuracy: 0.9581 - val_loss: 0.6582 - val_binary_accuracy: 0.7500
Epoch 62/100
50/50 [==============================] - 33s 663ms/step - loss: 0.4278 - binary_accuracy: 0.9706 - val_loss: 0.7351 - val_binary_accuracy: 0.6250
Epoch 63/100
50/50 [==============================] - 33s 665ms/step - loss: 0.4263 - binary_accuracy: 0.9700 - val_loss: 0.6595 - val_binary_accuracy: 0.7500
Epoch 64/100
50/50 [==============================] - 33s 662ms/step - loss: 0.4234 - binary_accuracy: 0.9719 - val_loss: 0.6198 - val_binary_accuracy: 0.8125
Epoch 65/100
50/50 [==============================] - 33s 663ms/step - loss: 0.4311 - binary_accuracy: 0.9650 - val_loss: 0.7357 - val_binary_accuracy: 0.6250
Epoch 66/100
50/50 [==============================] - 33s 669ms/step - loss: 0.4267 - binary_accuracy: 0.9594 - val_loss: 0.5834 - val_binary_accuracy: 0.8750
Epoch 67/100
50/50 [==============================] - 34s 6

In [ ]:
final_accuracy = model.evaluate_generator(test_generator,624)[1]

In [ ]:
final_accuracy

0.9038461446762085